# Introduction 

We analyze data from a two-alternative forced choice task (e.g. the random-dot-task). In these tasks ambiguous evidence for two alternative choices is presented to an observer. The ambiguity(歧义) results in imperfect performance, that varies with the strength of the ambiguity. This relationship is quantified by the "psychometric(心理测量学) function".

## Prerequisites

- two-alternative forced choice task
- the psychometric function relates an observer's performance to the observed stimulus
- basic Python skills


## Objectives

- perform elementary data analysis steps on real data
- calculate a psychometric function from data

## Notes

All problems can be solved with either numpy or pandas

# Notebook setup 

In [1]:
import numpy as np
import time 
import matplotlib.pyplot as plt

# Problem 1: Monkey dot data

## Load data

** Instruction **

Load the data. Note:

- Read the data from file 'dots_psychophysics.txt'
- Each line is one trial, the columnns encode
    - coherence of random dot pattern
    - direction of random dot pattern
    - the direction the monkey chose
    - if the monkey was rewarded
    - the monkey's reaction time
- direction / choice is encoded as: 1 = 0% coherence, 2 = left, 3 = right
- on 0% coherence trials the monkey was rewarded randomly

In [19]:
data_in = np.loadtxt('./data/dots_psychophysics.txt')
data_in[:30]

array([[1.280000e-01, 2.000000e+00, 2.000000e+00, 1.000000e+00,
        6.226750e+02],
       [0.000000e+00, 1.000000e+00, 1.000000e+00, 1.000000e+00,
        9.344500e+02],
       [0.000000e+00, 1.000000e+00, 1.000000e+00, 1.000000e+00,
        8.374750e+02],
       [5.120000e-01, 2.000000e+00, 2.000000e+00, 1.000000e+00,
        3.495000e+02],
       [2.560000e-01, 2.000000e+00, 2.000000e+00, 1.000000e+00,
        4.159250e+02],
       [3.200000e-02, 2.000000e+00, 1.000000e+00, 0.000000e+00,
        5.337750e+02],
       [7.240000e-01, 2.000000e+00, 2.000000e+00, 1.000000e+00,
        3.114500e+02],
       [0.000000e+00, 1.000000e+00, 2.000000e+00, 1.000000e+00,
        8.699250e+02],
       [5.120000e-01, 2.000000e+00, 2.000000e+00, 1.000000e+00,
        2.825500e+02],
       [6.400000e-02, 2.000000e+00, 2.000000e+00, 1.000000e+00,
        4.049000e+02],
       [3.200000e-02, 3.000000e+00, 1.000000e+00, 0.000000e+00,
        4.758750e+02],
       [5.120000e-01, 3.000000e+00, 3.00000

*** NOTE: ***

- direction == 1 encodes 0% coherence stimuli
- direction == 2 encodes 'left' stimuli
- direction == 3 encodes 'right' stimuli

## Consistency check 

**Instruction**

- check that whenever coherence is 0, direction also encodes the 0% stimulus

In [3]:
start_time = time.time()
data_dirs = data_in[:,1]
data_corh = data_in[:,0]
data_dirs_bin = np.zeros_like(data_dirs)
idx1 = np.where(data_dirs==1)
idx1_array = idx1[0]
print(idx1_array)
idx2 = np.where(data_corh==0)
idx2_array =  idx2[0]
print(idx2_array.shape)
zeros_extend = np.zeros(abs(idx1_array.shape[0] - idx2_array.shape[0]))
idx2_array_extend = np.concatenate((idx2_array,zeros_extend))#记住np.cancatenate()中要将连接的两个矩阵用括号给连接起来
print('idx2_array_extend:',idx2_array_extend[:20])
print('idx1_array:',idx1_array[:20])
print(idx2_array_extend.shape)
use_list = []
for j in range(idx2_array.shape[0]):
    item  = idx2_array_extend[j] 
    for i in range(idx1_array.shape[0]):
        if idx1_array[i] == item:
            use_list.append(item)
print(use_list[:20])
print('spend time:',time.time()-start_time)

[    1     2     7 ... 49455 49456 49462]
(8465,)
idx2_array_extend: [  1.   2.   7.  17.  19.  22.  43.  44.  51.  58.  63.  71.  72.  79.
  89.  90.  91.  94. 101. 105.]
idx1_array: [ 1  2  7 13 16 17 18 19 22 29 31 33 36 40 41 43 44 49 51 54]
(22093,)
[1.0, 2.0, 7.0, 17.0, 19.0, 22.0, 43.0, 44.0, 51.0, 58.0, 63.0, 71.0, 72.0, 79.0, 89.0, 90.0, 91.0, 94.0, 101.0, 105.0]
spend time: 43.83499240875244


**Instruction**

- what's the distribution of choices when coherence == 0?

In [4]:
idx2 = np.where(data_corh==0)
idx2_array =  idx2[0]
data_choice = data_in[:,2]
data_choice_cor_0 = data_choice[idx2_array]
num_total = data_choice_cor_0.shape
print("p of dir1:",data_choice_cor_0[data_choice_cor_0==1].sum()/1/num_total)
print("p of dir2:",data_choice_cor_0[data_choice_cor_0==2].sum()/2/num_total)
print('p of dir3:',data_choice_cor_0[data_choice_cor_0==3].sum()/3/num_total)

data_choice_cor_0.shape
#为均匀分布

p of dir1: [0.34022445]
p of dir2: [0.32746604]
p of dir3: [0.33230951]


(8465,)

**Instruction**

- For non-zero-coherence trials, is it true that whenever direction == choice then rewarded == True?

In [22]:
data_reward = data_in[:,-2]
data_corh_not_0 = data_corh[data_corh!=0]
idx3 = np.where(data_corh!=0)
idx3_array = idx3[0]
print(idx3_array[:30])
data_corh_not_0_dir = data_dirs[idx3]
print(data_corh_not_0_dir[:30])
data_corh_not_0_choice = data_choice[idx3]
print(data_corh_not_0_choice[:30])
count_not_equal = 0
for i in range(data_corh_not_0_dir.shape[0]):
    if data_corh_not_0_choice[i] == data_corh_not_0_dir[i]:
        if data_reward[idx3_array[i]] == 0:
            print("**")
        print(f"num:{i+1}\t,idx3:{idx3_array[i]}\t,reward:{data_reward[idx3_array[i]]}")
#当direction == choice then rewarded == True不一定成立

[ 0  3  4  5  6  8  9 10 11 12 13 14 15 16 18 20 21 23 24 25 26 27 28 29
 30 31 32 33 34 35]
[2. 2. 2. 2. 2. 2. 2. 3. 3. 3. 1. 3. 3. 1. 1. 2. 2. 2. 3. 2. 3. 2. 3. 1.
 2. 1. 3. 1. 2. 3.]
[2. 2. 2. 1. 2. 2. 2. 1. 3. 3. 1. 3. 3. 1. 1. 2. 3. 2. 3. 2. 3. 2. 3. 1.
 2. 1. 2. 1. 2. 3.]
num:1	,idx3:0	,reward:1.0
num:2	,idx3:3	,reward:1.0
num:3	,idx3:4	,reward:1.0
num:5	,idx3:6	,reward:1.0
num:6	,idx3:8	,reward:1.0
num:7	,idx3:9	,reward:1.0
num:9	,idx3:11	,reward:1.0
num:10	,idx3:12	,reward:1.0
num:11	,idx3:13	,reward:1.0
num:12	,idx3:14	,reward:1.0
num:13	,idx3:15	,reward:1.0
num:14	,idx3:16	,reward:1.0
num:15	,idx3:18	,reward:1.0
num:16	,idx3:20	,reward:1.0
num:18	,idx3:23	,reward:1.0
num:19	,idx3:24	,reward:1.0
num:20	,idx3:25	,reward:1.0
**
num:21	,idx3:26	,reward:0.0
num:22	,idx3:27	,reward:1.0
num:23	,idx3:28	,reward:1.0
num:24	,idx3:29	,reward:1.0
num:25	,idx3:30	,reward:1.0
num:26	,idx3:31	,reward:1.0
num:28	,idx3:33	,reward:1.0
num:29	,idx3:34	,reward:1.0
num:30	,idx3:35	,reward:1.0
num

* Does this make sense?
* Compare the reaction times for non-rewarded trials, in which the monkey chose the stimulus direction to those of other trials

## How much data do we have, broken down by direction?

** Instruction **

Make a bar plot of the number of trials, broken down by stimulus direction and coherence. Don't forget axes labels and a plot title.

## Psychometric function

** Instruction **

Plot the psychometric function, i.e. fraction of correct choices vs. coherence

- once collapsing all directions
- once keeping directions (i.e. plot fraction of right choices for all coherence levels, where for left choices coherence is set to negative values)

### Aborted trials

Filter out aborted trials, i.e. trials in which the monkey chose the stimulus direction but wasn't rewarded

### Use all directions

### Use only direction == 'left' and direction == 'right' trials

# [Small project] Rat odor data

## Load data

** Instructions **

- load the file "data/odor_psychophysics.csv'
- the columns represent: 'rat', 'session', 'direction', 'contrast', 'choice', 'rewarded'
- analyze the data
- in particular
    - plot the psychometric curve for each rat
    - plot the mean psychometric curve for all rats including error bars